In [325]:
# All necessary imports here
import oci
import os.path
import sys
import json
import logging
import pprint
import re
from collections import Counter 
import ipaddr #pip3 install ipaddr

In [326]:
# Read config and create clients (identity,network,etc.)

config = oci.config.from_file()
identity_client = oci.identity.IdentityClient(config)
virtual_network_client = oci.core.VirtualNetworkClient(config)
virtual_network_composite_operations = oci.core.VirtualNetworkClientCompositeOperations(virtual_network_client)

In [327]:
# local logger
def local_logger(value):    
    print(value)

In [328]:
def print_decorator(message):
    print ("====================")
    print (message)
    print ("====================")

In [329]:
# Helper methods for extracting and json_lookup
def extract_value_by_field(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results


In [330]:
# helper method to convert response to dictionary
def convert_response_to_dict(oci_response):
    return oci.util.to_dict(oci_response.data)

In [331]:
# Special Characters Regex validation
def special_char_regex_validation(value):
        special_chars_regex = re.compile('[@!#$%^&*()<>?/\|}{~:`]')
        special_char_check = special_chars_regex.search(value)
        if special_char_check is None:            
            return True
        else:
            return False

In [332]:
# IPV4 CIDR Notation Regex Validation        
def ipv4_regex_validation(value):
    ipv4_cidr_regex = re.compile('^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])/(1[0-9]|2[0-9]|3[0-1])$')    
    ipv4_cidr_check = ipv4_cidr_regex.search(value)
    if ipv4_cidr_check is None:
        return False
    else:
        return True

In [333]:
# Json file exist validation
def validate_file_exist():
    if os.path('./hub_spokes.json').is_file():
        local_logger ("====================")
        local_logger ("Input file hub_spokes.json exists !")
        local_logger ("====================")
        return True
    else:
        local_logger ("====================")
        local_logger ("Input file hub_spokes.json does not exists !")
        local_logger ("====================")
        return False

In [334]:
# Json file complete payload validation
def validate_json(payload):
    # Name sanitization check i.e. all vcn name and dns-name and drg-name   
    hub_check = False
    spoke_check = False
    hub_vcn = payload["hub"]["vcn"]
    hub_drg = payload["hub"]["drg"]
    
    spokes = payload["spokes"]
    spoke_vcns = []
    
    for spoke in spokes:
        spoke_vcns.append(spoke["vcn"])
        
    hub_vcn_name = special_char_regex_validation(hub_vcn["name"])
    hub_vcn_dns_name = special_char_regex_validation(hub_vcn["dns_name"])
    
    hub_drg_name = special_char_regex_validation(hub_drg["name"])
    
    if hub_vcn_name and hub_vcn_dns_name and hub_drg_name:
        hub_check = True
    else:
        hub_check = False
    
    for spoke in spoke_vcns:
        spoke_vcn_name = special_char_regex_validation(spoke["name"])
        spoke_vcn_dns_name = special_char_regex_validation(spoke["dns_name"])
        if spoke_vcn_name and spoke_vcn_dns_name:
            spoke_check = True
        else:
            spoke_check = False
            break
    
    if hub_check and spoke_check:
        local_logger("============SUCCESS=======")
        local_logger("HUB AND SPOKE CHECK FOR SPECIAL CHARACTER PASSED !")
        local_logger("==========================")
    
    else:
        local_logger("===========FAILED=========")
        local_logger("HUB AND SPOKE CHECK FOR SPECIAL CHARACTER FAILED !")
        local_logger("==========================")
    
    # CIDR overlap check VCN level
    hub_cidr_blocks = hub_vcn["cidr"]
    cidr_blocks = []
    
    cidr_blocks.append(ipaddr.IPNetwork(hub_cidr_blocks))

    for spoke_vcn in spoke_vcns:
        cidr_blocks.append(ipaddr.IPNetwork(spoke_vcn["cidr"]))

    
    for index in range(len(cidr_blocks)):        
        for ind in range(len(cidr_blocks)):
            if index!=ind:
                is_overlap = cidr_blocks[index].overlaps(cidr_blocks[ind])
                if is_overlap is True:
                    hub_check = False
                    spoke_check = False                    
                    break
                else:
                    hub_check = True
                    spoke_check = True
                    
    
    if hub_check and spoke_check:
        local_logger("==========SUCCESS=========")
        local_logger("CIDR BLOCKS CHECK IN THE INPUT HUB/SPOKE SUCCESSFULLY VALIDATED !" )
        local_logger("==========================")
    else:
        local_logger("=========FAILED===========")
        local_logger("OVERLAPPING CIDR BLOCKS FOUND IN THE INPUT HUB/SPOKE !" )
        local_logger("==========================")
        
    return True if hub_check and spoke_check else False
    

In [335]:
# Compartment validation

def fetch_all_compartments_in_tenancy(client):
    """Fetch all Compartments in Tenancy , and look across all subtrees."""
    compartmentResponse = oci.pagination.list_call_get_all_results(
        client.list_compartments,
        compartment_id=tenancy_ocid,
        limit=200,
        access_level="ACCESSIBLE",
        compartment_id_in_subtree=True,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
    )
    return convert_response_to_dict(compartmentResponse)

def checkByOCID_if_compartment_active(compartment_id, client):
    compartment_response = client.get_compartment(
        compartment_id=compartment_id, retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
    )
    compartment = convert_response_to_dict(compartment_response)
    return True if compartment["lifecycle_state"] == "ACTIVE" else False

def filter_compartments_by_state(compartmentList=[], compartmentState="ACTIVE"):
    """Filter Compartments by their lifecycle state, ACTIVE| DELETNG | DELETED | CREATING"""
    filteredCompartments = [
        compartment
        for compartment in compartmentList
        if compartment["lifecycle_state"] == compartmentState
    ]
    return filteredCompartments

In [336]:
# VCN Checks

# Check if VCN created matches by Name
def checkVcnNameMatch(client, compartment_id, vcn_name):
    listVCNReponse = client.list_vcns(compartment_id = compartment_id)
    vcns = convert_response_to_dict(listVCNReponse)
    vcn_names = extract_value_by_field(vcns, "display_name")
    if vcn_name in vcn_names:
        print_decorator("VCN NAME ALREADY EXIST. SKIPPING VCN CREATION")
        return True
    else:
        return False
    

# Get VCN OCID for matched VCN Name
def checkVCNNameGetOCID(client, compartment_id, vcn_name):
    listVCNReponse = client.list_vcns(compartment_id = compartment_id)
    vcns = convert_response_to_dict(listVCNReponse)    
    for vcn in vcns:
        if vcn["display_name"]==vcn_name:
            return vcn["id"]
    else:
        return None
    
# Get VCN OCID for matched VCN CIDR
def checkVCNCidrGetOCID(client, compartment_id, cidr):
    listVCNReponse = client.list_vcns(compartment_id = compartment_id)
    vcns = convert_response_to_dict(listVCNReponse)    
    for vcn in vcns:
        if vcn["cidr_block"]==cidr:
            return vcn["id"]
    else:
        return None

# Check if VCN is in available state
def checkVCNStateAvailable(client, compartment_id, vcn_id):
    getVCNResponse = client.get_vcn(vcn_id = vcn_id)
    vcn = convert_response_to_dict(getVCNResponse)
    if vcn["lifecycle_state"] == "AVAILABLE":
        return True
    else:
        return False
            
# Check if VCN created matches by CIDR
def checkVcnCidrMatch(client, compartment_id, vcn_cidr):
    getResponse = client.list_vcns(compartment_id = compartment_id)
    vcns = convert_response_to_dict(getResponse)
    for vcn in vcns:
        if vcn["cidr_block"] == vcn_cidr:                        
            return True
        else:
            return False
    
# Get VCN by VCN OCID
def getVCNbyOCID(client, compartment_id, vcn_ocid):
    try:
        getVCN = client.get_vcn(vcn_id = vcn_ocid)
        vcn = getVCN.data
        return vcn.id
    except Exception as e:
        return None
        
getVCNbyOCID(virtual_network_client, config["compartment"], "ocid1.vcn.oc1.phx.amaaaaaa43cggciamaezwaggscjbvqgbwzbo4zlakrisgc4xrnkh2k6kbzga")

'ocid1.vcn.oc1.phx.amaaaaaa43cggciamaezwaggscjbvqgbwzbo4zlakrisgc4xrnkh2k6kbzga'

In [343]:
# Create VCN
def create_new_vcn(client,composite_client,  vcn_object):
    vcn_details = oci.core.models.CreateVcnDetails(cidr_block=vcn_object["cidr"],
        display_name=vcn_object["name"],
        compartment_id=vcn_object["compartment_id"],
        dns_label= vcn_object["dns_name"])
    VCNResponse = composite_client.create_vcn_and_wait_for_state(
    vcn_details, 
        wait_for_states=[oci.core.models.Vcn.LIFECYCLE_STATE_AVAILABLE]
    )
    vcn = convert_response_to_dict(VCNResponse)
    return vcn["id"]

def check_before_vcn_create(client,composite_client, compartment_id, vcn_object):
    vcn_ocid = None
    checkCompartmentActive = checkByOCID_if_compartment_active(compartment_id, identity_client)
    print_decorator("COMPARTMENT EXISTS AND ACTIVE !") if checkCompartmentActive else print_decorator("COMPARTMENT DOES NOT EXIST OR IS NOT ACTIVE !")
    
    checkVCNNameMatch = checkVcnNameMatch(client, compartment_id, vcn_object["name"])
    
    if checkVCNNameMatch:
        vcn_ocid = checkVCNNameGetOCID(client, compartment_id, vcn_object["name"])
    
    checkVCNCIDRMatch = checkVcnCidrMatch(client, compartment_id, vcn_object["cidr"])
    
    if checkVCNCIDRMatch:
        vcn_ocid = checkVCNCidrGetOCID(client, compartment_id, vcn_object["cidr"])
    
    if vcn_ocid is None:
        return create_new_vcn(client,composite_client, vcn_object)
    else:
        print_decorator("VCN ALREADY EXIST ! SKIPPING VCN CREATION !")
        return vcn_ocid
    
    

In [344]:
# DRG Checks

# Check if DRG exist by name
def is_drg_exist_by_name(client, compartment_ocid, drg_name):
    listDRGs = client.list_drgs(compartment_id = compartment_ocid)
    drgs = convert_response_to_dict(listDRGs)
    drg_name_extract = extract_value_by_field(drgs, "display_name")
    return True if drg_name in drg_name_extract else False

# Get DRG OCID from DRG Name
def get_drg_ocid_by_name(client, compartment_ocid, drg_name):
    listDRGs = client.list_drgs(compartment_id = compartment_ocid)
    drgs = convert_response_to_dict(listDRGs)
    for drg in drgs:
        if drg["display_name"] == drg_name:
            return drg["id"]
        else:
            return None
        
# Check if DRG exist already by OCID and State in Compartment        
def is_drg_already_exist_in_compartment(client, compartment_ocid, drg_ocid):
    drg = client.get_drg(drg_id = drg_ocid, retry_strategy= oci.retry.DEFAULT_RETRY_STRATEGY)
    drg_dict = convert_response_to_dict(drg)
    if drg_dict is not None and drg_dict["lifecycle_state"] == "AVAILABLE":
        return True
    else:
        return False   
    
# Check DRG attachment status 
def get_drg_attachment_status(client, drg_attachment_ocid):
    drg_attachment = client.get_drg_attachment(drg_attachment_id = drg_attachment_ocid)
    drg_attachment_dict = convert_response_to_dict(drg_attachment)
    if drg_attachment_dict is not None and drg_attachment_dict["lifecycle_state"]=="ATTACHED":
        return True
    else:
        return False
    
def get_drg_attachment_ocid(client, drg_attachment_ocid):
    drg_attachment = client.get_drg_attachment(drg_attachment_id = drg_attachment_ocid)
    drg_attach = convert_response_to_dict(drg_attachment)
    return drg_attach["id"]

def filter_drg_attachment_id(client,compartment_ocid, drg_id):
    drg_attachments = client.list_drg_attachments(compartment_id = compartment_ocid)
    drg_attachments_dict = convert_response_to_dict(drg_attachments)
    for drg_attachment in drg_attachments_dict:
        if drg_attachment["drg_id"] == drg_id:
            return drg_attachment["id"]
            break
        else:
            return None


In [345]:
# Create DRG
def create_new_drg(client, compartment_id, hub_drg):
    drg_result = client.create_drg(
        oci.core.models.CreateDrgDetails(
            compartment_id=compartment_id,
            display_name= hub_drg["name"]
        )
    )
    drg = oci.wait_until(
        client,
        client.get_drg(drg_result.data.id),
        'lifecycle_state',
        'AVAILABLE'
    )
    local_logger('Created DRG')
    local_logger('===============')
    local_logger('\n')
    drg_new = convert_response_to_dict(drg)
    return drg_new["id"]

# Create DRG if drg does not exist already
def check_before_drg_create(client, compartment_id, hub_drg):
    drg_ocid = None
    checkDRGStatusCompartment = False
    checkCompartmentActive = checkByOCID_if_compartment_active(compartment_id, identity_client)
    print_decorator("COMPARTMENT EXISTS AND ACTIVE !") if checkCompartmentActive else print_decorator("COMPARTMENT DOES NOT EXIST OR IS NOT ACTIVE !")
    checkDRGNameMatch = is_drg_exist_by_name(client, compartment_id, hub_drg["name"])
    
    if checkDRGNameMatch:
        drg_ocid = get_drg_ocid_by_name(client, compartment_id, hub_drg["name"])
        checkDRGStatusCompartment = is_drg_already_exist_in_compartment(client, compartment_id, drg_ocid)
    
    if drg_ocid is None and checkDRGStatusCompartment is False:
        return create_new_drg(client,compartment_id,hub_drg)
    else:
        print_decorator("DRG ALREADY EXIST ! SKIPPING DRG CREATION !")
        return drg_ocid
    
    
# Attach newly created DRG to VCN using VCN OCID
def drg_attach(client, vcn_ocid, drg_ocid, drg_name):
    drg_attach_result = client.create_drg_attachment(
        oci.core.models.CreateDrgAttachmentDetails(
            display_name= drg_name,
            vcn_id=vcn_ocid,
            drg_id=drg_ocid
        )
    )
    drg_attachment = oci.wait_until(
        client,
        client.get_drg_attachment(drg_attach_result.data.id),
        'lifecycle_state',
        'ATTACHED'
    )
    local_logger('Created DRG Attachment')
    local_logger('=========================')
    local_logger('\n')
    drg_attach = convert_response_to_dict(drg_attachment)
    return drg_attach["id"]

def check_before_drg_attach(client, compartment_id, vcn_ocid, drg_ocid, drg_name):
    drg_attachment_ocid = None
    checkCompartmentActive = checkByOCID_if_compartment_active(compartment_id, identity_client)    
    print_decorator("COMPARTMENT EXISTS AND ACTIVE !") if checkCompartmentActive else print_decorator("COMPARTMENT DOES NOT EXIST OR IS NOT ACTIVE !")
    checkIfVCNIsAvailable = checkVCNStateAvailable(client, compartment_id, vcn_ocid)
    checkIfDRGIsAvailable = is_drg_already_exist_in_compartment(client, compartment_id, drg_ocid)
    drg_attachment_ocid = filter_drg_attachment_id(client, compartment_id, drg_ocid)
    
    if drg_attachment_ocid is not None:
        print_decorator("DRG ATTACHMENT ALREADY EXIST ! SKIPPING DRG ATTACHMENT !")
        return drg_attachment_ocid
    
    if checkCompartmentActive and checkIfVCNIsAvailable and checkIfDRGIsAvailable:
        return drg_attach(client, vcn_ocid, drg_ocid, drg_name)
        



In [346]:
# LPG METHODS

# Check for name match in LPG
def match_lpg_by_name(client, compartment_ocid,vcn_ocid, lpg_name):
    listLPGs = client.list_local_peering_gateways(compartment_id = compartment_ocid, vcn_id= vcn_ocid)
    lpgs = convert_response_to_dict(listLPGs)
    lpg_names = extract_value_by_field(lpgs, "display_name")
    if lpg_name in lpg_names:
        return True
    else:
        return False

# Get LPG OCID from matching name
def get_lpg_ocid (client, compartment_ocid,vcn_ocid, lpg_name):
    listlpgs = client.list_local_peering_gateways(compartment_id = compartment_ocid, vcn_id= vcn_ocid)
    lpgs = convert_response_to_dict(listlpgs)
    lpg_id= None
    for lpg in lpgs:
        if lpg["display_name"]==lpg_name:
            lpg_id = lpg["id"]
        else:
            lpg_id = None
    return lpg_id
        
        
# Check for LPG state
def is_lpg_available_status(client, lpg_ocid, compartment_ocid):
    lpg = client.get_local_peering_gateway(local_peering_gateway_id = lpg_ocid)
    lpg_dict = convert_response_to_dict(lpg)
    if lpg_dict["lifecycle_state"] == "AVAILABLE":
        return True
    else:
        return False
    
# Create Local Peering Gateway been Hub and Spokes VCN
def create_local_peering_gateway(composite_client, lpg_name, compartment_id, vcn_ocid):
    create_lpg_details = oci.core.models.CreateLocalPeeringGatewayDetails(compartment_id = compartment_id, display_name = lpg_name, vcn_id = vcn_ocid)
    create_lpg_response = composite_client.create_local_peering_gateway_and_wait_for_state(
    create_lpg_details,
    wait_for_states=[oci.core.models.LocalPeeringGateway.LIFECYCLE_STATE_AVAILABLE]
        )
    lpg = create_lpg_response
    lpg_dict = convert_response_to_dict(lpg)
    local_logger('Created LPG')
    local_logger('=========================')
    local_logger('\n')
    
    return lpg_dict["id"]


def check_before_lpg_create(client,composite_client, compartment_id, lpg_name, vcn_ocid):
    checkCompartmentActive = checkByOCID_if_compartment_active(compartment_id, identity_client)
    print_decorator("COMPARTMENT EXISTS AND ACTIVE !") if checkCompartmentActive else print_decorator("COMPARTMENT DOES NOT EXIST OR IS NOT ACTIVE !")    
    vcn_exist = getVCNbyOCID(client, compartment_id, vcn_ocid)
    if vcn_exist is not None:
        vcn_state = checkVCNStateAvailable(client, compartment_id, vcn_ocid)
    checkLpgNameMatch = match_lpg_by_name(client, compartment_id, vcn_ocid, lpg_name)
    getLpgocid = get_lpg_ocid(client, compartment_id,vcn_ocid, lpg_name)
    if getLpgocid is not None:
        getLpgStatus = is_lpg_available_status(client, getLpgocid, compartment_id)
    if checkLpgNameMatch and getLpgStatus:
        print_decorator("LPG EXIST ALREADY FOR THIS VCN")
        return getLpgocid
    else:
        return create_local_peering_gateway(composite_client, lpg_name, compartment_id, vcn_ocid)


In [349]:
# Quickstart code
def quick_start():
    with open('./hub_spokes.json') as hub_spokes:
        data = json.load(hub_spokes)
        # TODO: JSON Validation to be captured
        validate_json(data)
        vcn_ocid = check_before_vcn_create(virtual_network_client, virtual_network_composite_operations, config["compartment"], data["hub"]["vcn"])
        drg_ocid = check_before_drg_create(virtual_network_client, config["compartment"], data["hub"]["drg"])
        drg_attachment_ocid = check_before_drg_attach(virtual_network_client, config["compartment"], vcn_ocid, drg_ocid, data["hub"]["drg"]["name"])
        lpg_ocid = check_before_lpg_create(virtual_network_client, virtual_network_composite_operations, config["compartment"], "HubLPG", vcn_ocid)
        
        for spoke in data["spokes"]:
            vcn_ocid = check_before_vcn_create(virtual_network_client,virtual_network_composite_operations, config["compartment"], spoke["vcn"])
            lpg_ocid = check_before_lpg_create(virtual_network_client, virtual_network_composite_operations, config["compartment"], "SpokeLPG", vcn_ocid)
        
        

In [350]:
quick_start()

============SUCCESS=======
HUB AND SPOKE CHECK FOR SPECIAL CHARACTER PASSED !
==========SUCCESS=========
CIDR BLOCKS CHECK IN THE INPUT HUB/SPOKE SUCCESSFULLY VALIDATED !
COMPARTMENT EXISTS AND ACTIVE !
COMPARTMENT EXISTS AND ACTIVE !
DRG ALREADY EXIST ! SKIPPING DRG CREATION !
COMPARTMENT EXISTS AND ACTIVE !
DRG ATTACHMENT ALREADY EXIST ! SKIPPING DRG ATTACHMENT !
COMPARTMENT EXISTS AND ACTIVE !
Created LPG


{'vcn': {'cidr': '192.168.0.0/24', 'name': 'spoke1', 'compartment_id': 'ocid1.compartment.oc1..aaaaaaaad7ngb2s3armaa6qdkglst4nxnvvdrjjggq5afhfqxqgiyyoyluiq', 'dns_name': 'spoke1', 'subnets': []}}
COMPARTMENT EXISTS AND ACTIVE !
VCN NAME ALREADY EXIST. SKIPPING VCN CREATION
VCN ALREADY EXIST ! SKIPPING VCN CREATION !
COMPARTMENT EXISTS AND ACTIVE !
LPG EXIST ALREADY FOR THIS VCN
{'vcn': {'cidr': '192.168.1.0/24', 'name': 'spoke2', 'compartment_id': 'ocid1.compartment.oc1..aaaaaaaad7ngb2s3armaa6qdkglst4nxnvvdrjjggq5afhfqxqgiyyoyluiq', 'dns_name': 'spoke2', 'subnets': []}}
COMPARTME